In [2]:
! ipython kernel install --user --name=translational_variability_3.9

Installed kernelspec translational_variability_3.9 in /home/warri/.local/share/jupyter/kernels/translational_variability_3.9


In [9]:
! which python | echo > ./output.txt

In [1]:
from simalign import SentenceAligner
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-04-16 01:00:26,933 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [2]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

sentence = "I hate"
model.translate(sentence, target_lang='ru', beam_size=1)

/home/warri/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Ненавижу'

In [3]:
with open("./corpora/karenina/originals/Books_ru.txt", "rt", encoding="utf-8") as file:
    ru_corpus_raw = [line.rstrip() for line in file]

In [4]:
with open("./corpora/karenina/ru_tokens.txt", "rt", encoding="utf-8") as file:
    ru_tokens = [line.rstrip() for line in file]

In [8]:
! python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 966.6 kB/s eta 0:00:00m eta 0:00:010:00:07
You should consider upgrading via the '/home/warri/code/aaa/translational_variability/translational_variability_3.9/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [5]:
from tqdm import tqdm
import spacy

In [6]:
en_nlp_lg = spacy.load("en_core_web_lg")

In [12]:
mwmf = ""
itermax = ""
inter = ""
translation_text = ""
docs = []

for i in tqdm(range(len(ru_corpus_raw))):
    translation = model.translate(ru_corpus_raw[i], source_lang="ru", target_lang='en', beam_size=15, max_new_tokens=512)
    spacy_doc = en_nlp_lg(translation)
    en_tokens = [word.text for word in spacy_doc]
    docs.append(spacy_doc)
    translation_text += translation
    translation_text += '\n'

100%|███████████████████████████████████| 17255/17255 [5:11:02<00:00,  1.08s/it]


In [1]:
mwmf = ""
itermax = ""
inter = ""

for i, doc in tqdm(enumerate(docs)):
    en_tokens = [word.text for word in doc]
    print(ru_tokens[i], "\n", en_tokens)
    src, trg = (ru_tokens[i], en_tokens)
    alignments = myaligner.get_word_aligns(src, trg)
    mwmf += " ".join([f"{x}-{y}" for x, y in alignments["mwmf"]])
    itermax += " ".join([f"{x}-{y}" for x, y in alignments["itermax"]])
    inter += " ".join([f"{x}-{y}" for x, y in alignments["inter"]])
    mwmf += "\n"
    itermax += "\n"
    inter += "\n"

NameError: name 'tqdm' is not defined

In [17]:
with open('./mwmf.txt', 'wt') as f:
    f.write(mwmf)
with open('./itermax.txt', 'wt') as f:
    f.write(itermax)
with open('./inter.txt', 'wt') as f:
    f.write(inter)

In [14]:
with open('./translations_karenina_beam15.txt', 'wt') as f:
    f.write(translation_text)

In [ ]:
translation_list2 = []
for line in tqdm(ru_corpus_raw[5000:10000]):
    translation = model.translate(line, source_lang="ru", target_lang='en', beam_size=20)
    translation_list2.append(translation)

In [ ]:
translation_list3 = []
for line in tqdm(ru_corpus_raw[10000:]):
    translation = model.translate(line, source_lang="ru", target_lang='en', beam_size=20)
    translation_list3.append(translation)

In [18]:
from spacy.tokens import DocBin

doc_bin = DocBin()
for doc in docs:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

In [21]:
with open("./translated_spacy_doc_beam15.bin", "wb") as file:
    file.write(bytes_data)

In [ ]:
translation_data = ["" for i in range(len(ru_corpus_raw))]
aligned_data_itermax = ["" for i in range(len(ru_corpus_raw))]
aligned_data_inter = ["" for i in range(len(ru_corpus_raw))]
aligned_data_mwmf = ["" for i in range(len(ru_corpus_raw))]
for idx, translation, alignments in data[2]:
    translation_data[idx] = translation
    aligned_data_mwmf[idx] = " ".join([f"{x}-{y}" for x, y in alignments["mwmf"]])
    aligned_data_itermax[idx] = " ".join([f"{x}-{y}" for x, y in alignments["itermax"]])
    aligned_data_inter[idx] = " ".join([f"{x}-{y}" for x, y in alignments["inter"]])